# Dev space for data preparation codes

In [1]:
import xarray as xr

import glob

import dask

from src import data

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [3]:
PROCESSED_DATA_DIR = "../../data/processed/"

In [4]:
from dask.distributed import Client
from dask_jobqueue import PBSCluster

walltime = "02:00:00"
cores = 48
memory = "192GB"
cluster = PBSCluster(
    processes=1,
    walltime=str(walltime),
    cores=cores,
    memory=str(memory),
    job_extra=[
        "-q normal",
        "-l ncpus=" + str(cores),
        "-l mem=" + str(memory),
        "-P xv83",
        "-l jobfs=100GB",
        "-l storage=gdata/xv83+gdata/oi10+gdata/ua8",
    ],
    local_directory="$PBS_JOBFS",
    header_skip=["select"],
)

cluster.scale(jobs=1)
Client(cluster)

/g/data/xv83/users/ds0092/software/miniconda3/envs/squire_2022_correlation/lib/python3.10/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.6.53.30:8787/status,
Dashboard: http://10.6.53.30:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.53.30:46779,Workers: 0
Dashboard: http://10.6.53.30:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Prepare some data

In [5]:
_ = data.prepare_CanESM5(
    ["dcppA-hindcast", "dcppB-forecast"], 
    "Omon", 
    ["tos"],
    "tos_Omon_CanESM5_dcpp"
)

KeyboardInterrupt: 

In [12]:
xr.open_zarr(f"{PROCESSED_DATA_DIR}/psl_Amon_HadGEM3-GC31-MM_dcpp.zarr")

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360, init: 6, lead: 125, member: 10)
Coordinates:
    area     (lat, lon) float64 dask.array<chunksize=(20, 20), meta=np.ndarray>
  * init     (init) object 1960-11-16 00:00:00 ... 1965-11-16 00:00:00
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lead     (lead) int64 0 1 2 3 4 5 6 7 8 ... 117 118 119 120 121 122 123 124
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
  * member   (member) int64 1 2 3 4 5 6 7 8 9 10
    time     (init, lead) object dask.array<chunksize=(6, 125), meta=np.ndarray>
Data variables:
    psl      (init, member, lead, lat, lon) float32 dask.array<chunksize=(6, 10, 125, 20, 20), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [22]:
ds = data.prepare_NorCPM1_dcpp_variable("psl", "Amon")

In [24]:
ds

,Array,Chunk
Bytes,35.04 GiB,30.40 MiB
Shape,"(59, 20, 123, 180, 360)","(1, 1, 123, 180, 360)"
Count,14162 Tasks,1180 Chunks
Type,float32,numpy.ndarray


In [ ]:
data.prepare_CanESM5_dcpp_variable("tos", "Omon")

caught signal <cdo.Cdo.__getattr__.<locals>.Operator object at 0x149cb27e00d0> 2 <frame at 0x149cac2186f0, file '/g/data/xv83/users/ds0092/software/miniconda3/envs/squire_2022_correlation/lib/python3.10/threading.py', line 324, code wait>
caught signal <cdo.Cdo.__getattr__.<locals>.Operator object at 0x149cb27e00d0> 2 <frame at 0x149cac202de0, file '/g/data/xv83/users/ds0092/software/miniconda3/envs/squire_2022_correlation/lib/python3.10/threading.py', line 324, code wait>
caught signal <cdo.Cdo.__getattr__.<locals>.Operator object at 0x149cb27e00d0> 2 <frame at 0x149cac202de0, file '/g/data/xv83/users/ds0092/software/miniconda3/envs/squire_2022_correlation/lib/python3.10/threading.py', line 324, code wait>


In [113]:
data.prepare_HadGEM3_dcpp_variable("psl", "Amon")

In [4]:
data.prepare_HadSLP2r()

<xarray.Dataset>
Dimensions:  (lat: 37, lon: 72, time: 2040)
Coordinates:
    area     (lat, lon) float64 dask.array<chunksize=(37, 72), meta=np.ndarray>
  * lat      (lat) float32 90.0 85.0 80.0 75.0 70.0 ... -75.0 -80.0 -85.0 -90.0
  * lon      (lon) float32 0.0 5.0 10.0 15.0 20.0 ... 340.0 345.0 350.0 355.0
  * time     (time) datetime64[ns] 1850-01-01 1850-02-01 ... 2019-12-01
Data variables:
    slp      (time, lat, lon) float32 dask.array<chunksize=(2040, 37, 72), meta=np.ndarray>
Attributes:
    Conventions:    CF-1.0
    References:     https://hadobs.metoffice.com/gmslp/hadslp2/\nhttps://psl....
    comments:       adSLP2 was created using marine observations taken from I...
    dataset_title:  HadSLP2
    history:        Created 02/2006 by CAS
    institution:    Hadley Center
    platform:       Model
    source:         Hadley Center
    title:          HadSLP2 Real-Time

In [25]:
cluster.close()